## Import Dependencies

In [1]:
import pandas as pd
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Load Data

In [2]:
# Load the data
df = pd.read_csv('data\Bengali_hate_speech.csv')
df.head()

sentence  hate category
0                     যত্তসব পাপন শালার ফাজলামী!!!!!   1.0   sports
1                  পাপন শালা রে রিমান্ডে নেওয়া দরকার   1.0   sports
2  জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...   1.0   sports
3                শালা লুচ্চা দেখতে পাঠার মত দেখা যায়   1.0   sports
4   তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব   1.0   sports

## Check shape, Null value and Value_Count

In [3]:
df.shape

(19999, 3)

In [4]:
df.isnull().sum()

sentence        0
hate        10002
category        0
dtype: int64

In [5]:
df['hate'].value_counts()

1.0    4999
0.0    4998
Name: hate, dtype: int64

## Preprocessing

In [6]:
import spacy

In [7]:
nlp = spacy.blank("bn")

In [8]:
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.text)
    
    return " ".join(filtered_tokens) 

In [9]:
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'দেওয়ার', 'নেই', 'এখানেই', 'যতটা', 'যে', 'ঐ', 'তোর', 'সেই', 'ছাড়াও', 'দুটি', 'পারে', 'দ্বারা', 'তার', 'মোদের', 'হতে', 'হোক', 'যাঁরা', 'অন্তত', 'তা', 'তাহা', 'জনের', 'পাওয়া', 'হওয়ায়', 'পারি', 'নাগাদ', 'বলেন', 'চায়', 'সবার', 'যাওয়া', 'বলেছেন', 'ছাড়া', 'জানতে', 'বিনা', 'মাত্র', 'নয়', 'বসে', 'আবার', 'ধরে', 'জানায়', 'পর্যন্ত', 'জানা', 'ফের', 'হিসেবে', 'দেখা', 'গেছে', 'থেকেই', 'সময়', 'ওই', 'করি', 'তারপর', 'কয়েক', 'হবেন', 'ইত্যাদি', 'তাই', 'সেটাই', 'আগামী', 'গিয়েছে', 'উত্তর', 'ওকে', 'ছিল', 'এর', 'এঁদের', 'রয়েছে', 'অনেকেই', 'করায়', 'সেটাও', 'হয়ে', 'দিয়েছে', 'আপনার', 'তত', 'থেকে', 'গিয়েছে', 'সমস্ত', 'ফলে', 'যখন', 'করেছেন', 'মানুষ', 'যাঁর', 'উপরে', 'নানা', 'দেখতে', 'কয়েকটি', 'মোটেই', 'গেল', 'কিংবা', 'খুব', 'করিতে', 'হয়নি', 'বেশ', 'কেমনে', 'এত', 'সুতরাং', 'কত', 'তাতে', 'হওয়ায়', 'হৈলে', 'ছাড়াও', 'বলে', 'দিয়েছে', 'গিয়ে', 'দিয়েছেন', 'হতেই', 'দেওয়া', 'অনেকে', 'মনে', 'যাওয়ার', 'তাঁরা', 'আর', 'স্বয়ং', 'যথেষ্ট', 'একই', 'আপনি', 'রকম', 'হইয়া', 'করা', 'নেওয়া', 'হলেও', 'অবশ্য', 'পর', 'সেটা', 'গ

In [10]:
preprocess("যত্তসব পাপন শালার ফাজলামী!!!!! ৳")

'যত্তসব পাপন শালার ফাজলামী ৳'

In [11]:
df['preprocessed_sentence'] = df['sentence'].apply(preprocess) 

In [12]:
df.head()

sentence  hate category  \
0                     যত্তসব পাপন শালার ফাজলামী!!!!!   1.0   sports   
1                  পাপন শালা রে রিমান্ডে নেওয়া দরকার   1.0   sports   
2  জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...   1.0   sports   
3                শালা লুচ্চা দেখতে পাঠার মত দেখা যায়   1.0   sports   
4   তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব   1.0   sports   

                               preprocessed_sentence  
0                          যত্তসব পাপন শালার ফাজলামী  
1                        পাপন শালা রে রিমান্ডে দরকার  
2  জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ ভাবতে প...  
3                                  শালা লুচ্চা পাঠার  
4          শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব

In [13]:
import re

def remove_emojis(text):
    # Define a regex pattern for emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text).strip()

In [14]:
# Function to remove HTML tags
def remove_html_tag(text):
    br_pattern = re.compile(r'<\s*br\s*/?>,*,?')
    return br_pattern.sub('', text)

In [15]:
df['preprocessed_sentence'] = df['preprocessed_sentence'].apply(remove_emojis)

In [16]:
df['preprocessed_sentence'] = df['preprocessed_sentence'].apply(remove_html_tag)

In [17]:
df['sentence'][10406]

'মিরাজের খেলা ভালো লাগছে কার কার👇👇'

In [18]:
df['preprocessed_sentence'][10406]

'মিরাজের খেলা ভালো লাগছে কার কার'

In [19]:
df['sentence'][52]

'মাদারচুদ <br />শালা ভিক্ষারীর বাচ্চা<a href="http://www.youtube.com/results?search_query=%23admin">#admin</a>'

In [20]:
df['preprocessed_sentence'][52]

'মাদারচুদ শালা ভিক্ষারীর বাচ্চা < a href="http://www.youtube.com results?search_query=%23admin">#admin</a >'

## Seperate labeled and unlabeled data

In [21]:
df.head(3)

sentence  hate category  \
0                     যত্তসব পাপন শালার ফাজলামী!!!!!   1.0   sports   
1                  পাপন শালা রে রিমান্ডে নেওয়া দরকার   1.0   sports   
2  জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...   1.0   sports   

                               preprocessed_sentence  
0                          যত্তসব পাপন শালার ফাজলামী  
1                        পাপন শালা রে রিমান্ডে দরকার  
2  জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ ভাবতে প...

In [22]:
df = df.drop(['sentence', 'category'], axis=1)
df.head(3)

hate                              preprocessed_sentence
0   1.0                          যত্তসব পাপন শালার ফাজলামী
1   1.0                        পাপন শালা রে রিমান্ডে দরকার
2   1.0  জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ ভাবতে প...

In [23]:
# Separate labeled and unlabeled data
labeled_data = df.dropna(subset=['hate'])
unlabeled_data = df[df['hate'].isnull()].drop('hate', axis=1)

In [24]:
labeled_data.head(3)

hate                              preprocessed_sentence
0   1.0                          যত্তসব পাপন শালার ফাজলামী
1   1.0                        পাপন শালা রে রিমান্ডে দরকার
2   1.0  জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ ভাবতে প...

In [25]:
unlabeled_data.head(3)

preprocessed_sentence
4999  মিন্নি বুদার বিতরে এসিট মারার বিগার চুদার আয় দ...
5000                  কুত্তার বাচ্চারা ক্ষমতা পেলে পাগল
5001  শালার বেটারা শালীর বেটিরা সবকয়টা মাদারচোদ এগু...

## Seperate feature and terget of labeled data

In [26]:
# Split labeled data into features and labels
X_labeled = labeled_data['preprocessed_sentence']
y_labeled = labeled_data['hate']

In [27]:
X_labeled.head(3)

0                            যত্তসব পাপন শালার ফাজলামী
1                          পাপন শালা রে রিমান্ডে দরকার
2    জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ ভাবতে প...
Name: preprocessed_sentence, dtype: object

In [28]:
y_labeled.head(3)

0    1.0
1    1.0
2    1.0
Name: hate, dtype: float64

In [29]:
# Split labeled data into training and validation sets
# X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=42, stratify=y_labeled)
X_train, X_test, y_train, y_test = train_test_split(X_labeled, y_labeled, test_size=0.2, random_state=32, stratify=y_labeled)

In [30]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(7997,) (7997,) (2000,) (2000,)


## Build Model

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
# Create a base classifier (e.g., Decision Tree)
base_classifier = RandomForestClassifier(n_estimators=150, criterion = "entropy")

In [33]:
# Create a SelfTrainingClassifier
self_training_classifier = SelfTrainingClassifier(base_classifier)

In [34]:
#1. create a pipeline object
st_clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('st', self_training_classifier)         
])

In [35]:
# Initial training on labeled data
st_clf.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('st',
                 SelfTrainingClassifier(base_estimator=RandomForestClassifier(criterion='entropy',
                                                                              n_estimators=150)))])

In [36]:
# unlabeled_data = unlabeled_data['preprocessed_sentence']
# unlabeled_data

In [37]:
X_train.shape, unlabeled_data['preprocessed_sentence'].shape

((7997,), (10002,))

In [38]:
y_train.shape

(7997,)

In [39]:
X_test

14584    ভালোবাসার চাই খাটি প্রেমিক চাই ছেরে যাবেনা কোনদিন
14491    ধন্যবাদ শ্রদ্ধেয় কাজী হায়াৎ স্যারকে,,,মান্না...
4464                                               জানোয়ার
3311                                    লুইচ্ছার ফাঁসি চাই
11675                              শেষের মুহুতটা দেখার মতন
                               ...                        
3746               অফিসে সুন্দর সেক্সিদের চাকরি চুদতে পারা
14513                                      দেখলাম পরও খুশি
1404                                  পাপান কুত্তার বাচ্চা
12889    এটিএন মিউজিকের ব্যানারে উনার আরো তিনটা অডিও অ্...
13004                      ছবির গান গুলো লোড দিলে খুশি হবো
Name: preprocessed_sentence, Length: 2000, dtype: object

In [40]:
y_test[:15]

14584    0.0
14491    0.0
4464     1.0
3311     1.0
11675    0.0
10339    0.0
4547     1.0
12794    0.0
13803    0.0
1367     1.0
1986     1.0
14879    0.0
12292    0.0
12329    0.0
2653     1.0
Name: hate, dtype: float64

In [41]:
# Pseudo-labeling and iterative training
for iteration in range(10):
    # Pseudo-labeling on unlabeled data
    pseudo_labels = st_clf.predict(unlabeled_data['preprocessed_sentence'])

    # Combine labeled and pseudo-labeled data
    X_combined = pd.concat([X_train, unlabeled_data['preprocessed_sentence']], axis=0)
    y_combined = pd.concat([y_train, pd.Series(pseudo_labels)], axis=0)

    # Train the self-training classifier on the combined dataset
    st_clf.fit(X_combined, y_combined)
    
    y_pred = st_clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)
 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


0.667


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


0.6635


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


...

0.666


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


0.6695


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\semi_supervised\_self_training.py:210: UserWarning: y contains no unlabeled samples
  warnings.warn("y contains no unlabeled samples", UserWarning)


0.666


In [42]:
len(X_combined), len(y_combined)

(17999, 17999)

In [43]:
X_combined

3394               বিচি কেটে সালা গান্জাখুর দেখতেই মাতালের
12983            ভাই ভালো একটিং ভাই নাটক ভালো লাগে মজা পাই
2074     আল্লাহর গজব প্রকার এইটা খানকি মাগী জানে হেইর ব...
3252                      হালারা মেয়াদ উত্তির্ন গাজা খাইছে
4139     বয়স কম এখনো লুচ্চামি করস লজ্জা কিভাবে ছেলে মে...
                               ...                        
19994    লুচ্ছা হুজুরগুলোকে মুকি পাচাই লাতি মেরে দেশ তা...
19995                                   কুত্তাটা ফাঁসী দাও
19996          গিয়াসউদ্দিন এলাকায় খবর এনে মলম লাগায়ে লাগবো
19997                               হুজুর পিটিয়ে বের দরকার
19998    মহিলার মাথার চুল বৈশাখ মাসের খেরের পালা গরুকে ...
Name: preprocessed_sentence, Length: 17999, dtype: object

In [44]:
y_combined.shape

(17999,)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42, stratify=y_combined)

In [46]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

#1. create a pipeline object
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('rf', RandomForestClassifier(n_estimators=150, criterion = "entropy"))         
])

#2. fit with X_train and y_train
clf.fit(X_train, y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


#4. print the classfication report
print(classification_report(y_test, y_pred))

0.8319444444444445
              precision    recall  f1-score   support

         0.0       0.82      0.81      0.82      1653
         1.0       0.84      0.85      0.85      1947

    accuracy                           0.83      3600
   macro avg       0.83      0.83      0.83      3600
weighted avg       0.83      0.83      0.83      3600



In [47]:
j=0
for i in df['preprocessed_sentence']:
    print(j, i)
    j+=1

0 যত্তসব পাপন শালার ফাজলামী
1 পাপন শালা রে রিমান্ডে দরকার
2 জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ ভাবতে পারেনি।ধন্যবাদ তাহসিন ভাই
3 শালা লুচ্চা পাঠার
4 শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব
5 কুন দরনের কেলা ফাইজলামি তাস্কিন রে চর মারা দরকার
6 পাপন ভর মাদা চোদ পাপনে পদতেক চাই
7 দুরো সালার পুদ চুপথাক
8 কুত্তার বাছচা পাপন
9 বাল ছাল তর সাউয়া
10 কপালে জুতা মারি শালার পুত
11 পাপনে পাগল হয়াছে
12 দেখেছো মাগি হিজরা বোঝা হিজরা
13 হালার পু পাপন
14 শালায় কারণেই খেলায় ডাব্বা মারে
15 ব্ব
16 পাপন হালারে বেট জুতার মালা দিয়এই বের কেনে কারনে ভাল পিলিআর গুলো খেলা পারেনা
17 পাপন মাঘির পুতের পেটের মদ্দে দুর্নিতি ভরা
18 পাপনকে জুতার মালা দেয়া জরুর
19 জারসি পড়া লোকটাকে চুদেদিতাম
20 ইয়েছির আলী হালা খেলা জানেনা
21 কুত্তার বাচ্চা পাপন
22 পাপন মাগির পোলার
23 মাগির পুয়া
24 ক্রিকেট ওনার বাবার সম্পত্তি হালার ফু
25 বিসিবির সবকটাই চোরমাদারচোদ
26 কুকুরের চাচ্চা তদেক হয়চে নবীকে গারি দিয়াচে তদেক কিচু সাদারন মানুষকে গারিদিয়েচে তদেল মাতা বেতা বেরে
27 মুসফিক চেডের বাল
28 পাপন খেলব?কি হেকার ছাইলে পাপনকে মাগির দলাল বান

1317 পাপন খানকির পুলা বারে সক্ট হচ্ছিস কেনো তকে সাকিব বেপার টা যানিয়েছে চেপে বলছিস কেনো ভারত জুয়ারি দের তর হাত
1318 মাদার চুদ আরো আগায়া নিবো দেকবো কেমন আগায়া
1319 কানকির পুলারা
1320 ফুটপাতের টোকাইগুলা টাকাকড়ি সম্মান পায় lol
1321 পাপন গং দের পাছায় লাতি মেরে বাহির
1322 শালা জাউরাতুই মাসে লাখ টাকা বেতন পাসওটা
1323 খারাপ দেশর খেলোয়া ১০ খেলে ১০০করতে পারেনি ১৮ খেলে পারেনি
1324 চুতিয়া পাপনের পদত্যাগ চাই
1325 তাসকিনের বলার বাদ দেওয়ায় বাংলাদেশ টিমের অবস্থা ম্যাচ হারতাছে সালারা
1326 পাপন পদতেগ চাই বাইনচোদ পাপন সুনলে গা জ্বলে মন তোরে কম মিনিট গালি দেই
1327 পাপন হালা চিন্তায়কালো
1328 বাল খেলেতাদের টাকা দিবে
1329 কুত্তারবাচ্চা রুবেল গান গাইলো কখন
1330 সাংবাদিক ভাই সাকিবের ডান পাসের কুততা পাপোনের ছোবি কেনো তুলছো
1331 হালায় পুরা গাজা খোর
1332 পাপন বিচার আল্লাহ শালা করলিআর বাংলাদেশ ক্রিকেট ধ্বংস দিলি
1333 শখ বউয়ের ভোদায় দে।।ফুটবল শেষ ক্রিকেট টা ধংসের যাইতেছে শুয়েরের বাচ্চারা ভারতের সাতে খেলিস পরিবার দেখবে।।সোনার বাংলাদেশ রইলো দেশে বসবাস জিন্দা মরা
1334 পাপন খানকির পোলা।তোর আওয়ামী লীগের গুষ্টি চুদি তো

2376 মাদার চোদ জুতা পিটা
2377 সালমান শাহ বেচে থাকলে বাংলাদেশে সিনেমা হলের সংখ্যা 3000 আপামর জনগন সিনেমা দাড়িয়ে দাড়িযে সালমান শাহ’র ছবি দেখত।ঘাড় বেকা হিজড়া ভোদাই খান সাকিব জনপ্রিয় বাংলাদেশের 1,500 সিনেমা হলের সংখ্যা কমতে কমতে 175 রে দাড়িয়েছে।5বছর বাংলাদেশে সিনেমা না।সালমান শাহ’র নায়ক দেশে উদয় সিনেমা বাচবে
2378 সানিতা মাগি মাগি মাগি
2379 খানকি মাগির বাচ্চারা তোগোরে ফুটা কইরা ফালামু বালারে জাজ মাদারির বাচ্চারা ভালো অভিনয় মানে একদম জাজ গেছস বাল
2380 বালের মাসুদ রানা রে  অপমান মানে বুড়িয়া জাজ টা অপমান করছে।জাজরা নিজেরে
2381 সানাই মাদার তেরেসা বলছে প্রকাশ্য মানুষকে ভালবাসতে আল্লাহকে ভালবাসতে পারেনা মতন মাগীকে ভালবাসতে মাদার তেরেসাকে বলছে মানতে
2382 বাল বানাইছও
2383 বাংলাদেশের কতগুলো পাগল ছাগল বিচারকের আসনে বসাইছে বিচারকের আসনে বসিয়ে নাপিতের দিলে ভাল করতো জুতা মারা শুনলে মুখে কুকুরে প্রসাব
2384 আবাল গুলায় আনলাইক দিল
2385 মাগিকে সেফু দার লাইভে আসার দরকার
2386 বিচারকদের বলতাছি তদের সবাইকে চুদি
2387 হালার পো লাল পানজাবি টারে দেখলে মনডা ওরে মুগুর দিয়া পিডাই।হালা হুদাই হা হা হা গুরুর
2388 জুতা ম

3566 লুচ্চার বাচ্চার চুনু কেটে হাঁসকে খাওয়ানু
3567 সোনার বাংলাতে রক্ষক নামের ভক্ষকের জায়গা হবেনা।মাননীয় প্রধান মন্ত্রীর দৃষ্টান্তমূলক শাস্তি দাবি করছি।যেমন শালাকে শাস্তি অন্যায়
3568 বরগুনার কলেজের দাঁরানো মানুষগুলো আসলে হিজরা এতগুলা ছেলেকে এইভাবে মারল এগিয়ে গেলো হিজরা সালারা
3569 মাদার চুদ টাকা খাইছস মিন্নি মাগির বলার
3570 এরাই হাসিনার সোনার ছেলে হাসিনা মাগী ক্ষমতায়
3571 বিয়া করমু বলছে বিয়া করবা নানু ভাই আচ্ছা কর নানু ভাই নুনুটা পুটকির লাগেনি দেখ নানু ভাই চেক বলছে লাগেতো নানু ভাই লাগলে টা পুটকির থাক বিয়া সোদান লাগতনা
3572 বাইনচোদ এগুলো সালা অশিক্ষিত
3573 শালা কুওা জুতা মারেন সবাই তারে
3574 জানোয়ারদের ফাঁসি দেখার অপেক্ষায় আছি
3575 আপনাদের কান্না লাভ লাভ জানোয়ারদের সাজা পাওয়ার অবশ্যই দরকার জিন্দা পুড়িয়ে মারার সময় খেয়াল পরিবার ছেলে সন্তান মা বাবা ছেলে মেয়েরা কার পরিচয় রাস্তায় বের মীরজাফরের চেয়েও খারাপ
3576 বাংলাদেশ সরকার বাল ফালাবে।সরকারি আওয়ামিলিগ কুত্তা সরকার বাল ই ফালাবে
3577 শালির চেহারার অবস্তা
3578 হায়রে লুইচ্চার বাচ্চা
3579 জয় কুত্তালীগের
3580 পড়ার মাগিরা
3581 মাগি জানতো

4816 শালি এক্টা রেন্ডি হোটেলের মেয়ে
4817 নয়ন বনড বোনকে দিনের বেলা সকলের ল্যা‌‌ঙটা চুদা
4818 মাদার ছুদ কয়টার কারনে দেশ কোতায় বাজে জেলা প্রসাসক
4819 হায়রে বাংলাদেশের আইন এগুলো বালের আইন অন্যায় এখনো চাকরি কুত্তা লীগের বাচ্চার হাতে দেশ টা নষ্ট
4820 ভাই।এটা ইন্ডিয়ার চক্রান্ত খবর দেখবেন।তার পদে কনো মালাউন বশে এভাবেই সকল পদ কেড়ে নিবে আল্লাহ হেফাজত করুন
4821 খানকির পোলা রে মাইরা কুত্তা দিয়া খাওয়া নো হক
4822 সবাই পাছা ঢুকাও
4823 সালার গলা কেটে দাও রাইট
4824 ডিসি চুদলে নিলা চুদলে বিলা
4825 বেশিরভাগ সরকারি কর্মচারী কর্মকর্তারা অফিস আদালত যৌনপল্লীতে পরিণত দৃষ্টান্তমূলক শাস্তি দাবী করছি কুলাঙ্গার সুযোগ পায়
4826 খানকির ছেলেরা বাবে মারলো খানকির ছেলেদের দরে বাবে রিফাত রে কোপাইছে বাবে কোপানো উছিত
4827 মাদার চোদেরে পদতেক করাহক সরকারের অনুমেদন করছি
4828 সালিরে সালারে আগুনে পুড়ে মারা
4829 এগুলো বিসিএস ক্যাডার মাথা নুনু কন্ট্রোল দেশ কিভাবে কন্ট্রোল
4830 হেকারদের পাগলা কুত্তা কামরাইছে সালার কোয়া চেছি দিলে বাংলাদেশ টারে বরবাদ দিছে মাদার চোদেরা
4831 শালি সামুকের ডিসির ধনকে হেটা মুরাইছে ডিসি আরাম পাইছে মহি

...

14622 হাগার হাগার ভাই
14623 ফালতু সারমর্ম হীন নাটক।সময়ের অপচয়
14624 অপূর্ব বের নাটক পছন্দ
14625 ছবিটা মনের অজান্তেই চোখ পানি ঝরে পরলো  আসাধরন ছবি
14626 গল্প পড়ে বিশাল হাতি মরে পড়ে টা কুকুর হাতির মাংস খাচ্ছে কুকুর আরেকটা কামড়াচ্ছে ঝগড়া ছে একা খাবে আপনারাই দুইটা কুকুর মিলে মিশে মাংস খায় তবুও মাংস শেষ অবস্থা ছোট্র চ্যানেল সার্চক্রাইব অনুরোধ রইলো পতি
14627 আরেকটু ভালো খেলনা পিস্তল যেত
14628 খোব ভালো নাটকদেখে খোব ভালো লাগলো
14629 সাজ্জাদ ফাহামির পরতো প্রথমই বাদ পরতো
14630 সবায় চ্যালেন আনসাস্ক্রাইব করো এমবি খাই টাকাও খাই।তারা খুদার্থ ৫টাকা একদিন কিনলাম এমবি কাটে এপ ডাউনলোড করলাম রেজিস্ট্রার করলাম অধিক পরিমানে এমবি কাটে দূর বাল যত্যসব আজাইরা পাবলিক ফালতু     জান্নাত দেখার এতো করলাম
14631 মমর টা একটু
14632 বাপের টাকা মুভি চাই
14633 কথাগুলো দেখলে হাজারো এলোমেলো চিন্তারা ধমকে দাঁড়ায় কেন রে দুচোখে আসেনা ঘুম পলকে আদুরে চুপ জানিনা রে মন খুঁজে মন মনের মন
14634 ভাল্লাগলো
14635 তাহাসান বস
14636 মন চাইছে ফাহমি ভাই দুদ টিপ
14637 সুন্দর মুভি। কে দেখেছো লাইক
14638 কোথায় এসব পাগল ছাগল আসে
14639 সানি ভাইয়ের

16064 গুলোর ফাশি চাই
16065 আপলোড দিছে ধন্যবাদ সরকারি উচ্চ পরযায়ের কর্মচারী ওফিসে ভালো পারলো
16066 বাংলাদেশ আইন ভিডিও পুরা পোস্টে বোঝা যাইতেছে মিনি জড়িত বের হয়েও মিনি তাকায় কোথায় মিনি বাইকে উইঠা চইলা যাইতো হয়তোবা হতো বাংলাদেশের আইন এতই ঢিলা ঢালা বোঝেনা এখনই মিনি গ্রেফতার রিমান্ডে নেওয়া শালী মেন খুনি
16067 বুইড়া খাঁঠাস
16068 কয় হাস্যকর বিচার
16069 টা হুস থাকেনা
16070 মেয়েটির ফাঁসি চাই কান্না রাখতে পারিনি
16071 মনির সন্তানটির ডি এন এন টেস্ট করানো দরকার সন্তানটি সিরাজুদ দোল্লার আপনারা
16072 দেখুন এরপরে বেঁচে থাকতে এরকম নিকৃষ্ট প্রকাশ বাঁচতে মন কিভাবে খারাপ নির্লজ্জ গেছেন দেহ ব্যবসায় নামছে
16073 ওসি মোয়াজ্জেম ফাসি দেয়া
16074 রিফাতের স্ত্রী মিন্নি জড়িত সন্ধে মিন্নিকে রিমান্ডে নিলেই পরিস্কার
16075 অতি শিগগিরি ফাসি জনগণ
16076 সঠিক বিচার চাই
16077 মজা তুমিও পাইছো আমিও পাইছি লজ্জা থাকার জোর খুলতে খুলে দিলাম
16078 মিন্নি ফাঁসি শুনতে পাবো মিন্নি রিফাত নয়নের খেলায় মেতে উঠেছে ফাঁসি
16079 উন্নয়ন বাতাস লাগলে
16080 আপনাদের লাভ রিফাত হত্যা নয়ন হীর পুলিশ কুকুরের মারলও আপনেরা নাটক মানে আপনারা ট

16622 কোপ্পা সামচু
16623 মিন্নি হত্যার জড়িত
16624 সারে ওনাকে দুচে দিছে সারের দোষ নায়।আললাহ হেদায়েত দান করুক
16625 ভালো লগছে ভাই
16626 বোরকা পরলেই মেয়েটি ভালো দোষ ঈদের কোলাকুলি করেছিল
16627 আসলে গটনাটা মেয়ের ছেলে করতো দেশে দেশে নারী নির্যাতন মামলা হতো কাহনী করতো আসলে মেয়েটা রাস্তার পতিতালয় জন্ম ছিলো
16628 ভারতে এইরকম আইন তৈরী ভালো
16629 ভাই পুলিশ এখনো কিভাবে চুপ এইসব সিসিটিভি ফুটেজ বুঝিনা যতোটুকু বুঝলাম হত্যাকাণ্ডের মেয়েটি জড়িত
16630 মিস্টি বিতরন ফাঁসির
16631 সকল মা সন্তানের মমতা মেয়ে টাকে ছেলে মেরেছে নিমম চিন্তা মায়ের কতটা কষ্ঠ মায়ের জায়গা চিন্তা কুলাঙ্গার ছেলের পডবে
16632 চরিত্রহিনরা রাজনিতি ফাক বাংলাদেশ
16633 মন্ত্রীর মন্তব্য শুনে মুগ্ধ তদন্ত সিদ্ধান্ত নিবে মিনিটের বাকি এখনকি ডি এন টেস্ট
16634 স্যারের দোষ দোষ দোষ দেখছি আমাদেরকে সবাই মিলে বিচার
16635 মিননিকেও আইনের আওতায় আনা
16636 ধন্যবাদ আদালতকে
16637 গোসোল জাইতেছে আনটি
16638 পর্দাশীল মহিলা
16639 হিজাব অবসেসড
16640 জেনে কাহাকে খুনি বানানো না।এই খুনের এমপির ছেলে জরিত
16641 মা বাঁচার দরকার বাচ্চা থাকার পুরুষয়ের খারাপ চি চি
16642 আহ

17813 পুরো বাংলাদেশটাই পতিতালয়
17814 এরই নাম বাংলাদেশ ডিসি সাহেবের ভিডিও ডিসি সাহেব তদন্ত বাহ হাস্যকর কাহিনী আজকে ব্যক্তির ব্যক্তিটিকে হাজতে বড়া হইত মামলা হইত বাংলাদেশ উনিতো ডিসি সাহেব হবেনা বাংলাদেশ আইন সমান
17815 বলেছি ধরনের মিন্নি সংখা ধ্বংস হইতাছে রিফাতের জীবন মিন্নি
17816 ডিসি কেনো আওয়ামীলীগ উপরের নেতা মন্ত্রীরাই কর্মকান্ড বেড়ায়।জাস্ট সেগুলো ক্যামেরায় বন্দী এখনো
17817 হা হা হা হা হ্যাকের গ্রুপ
17818 সরকার আসলে ন্যায়পরায়ন অবশ্যই এটার সঠিক তদন্ত উল্লেখিত ডিসির শাস্তি প্রধান নজির সৃষ্টি জনগন আশাবাদী
17819 যতই পরিষ্কার ভিডিও প্রমান বের পুলিশের হাতে ধরিয়ে দেই এসব দিয়েও বাংলাদেশের আইনি ব্যবস্থা এতোটাই ঠুনকো পুলিশ কখনো মামলায় আসল অপরাধীদের শাস্তি পারেনি পারবেওনা
17820 স্যার ক্ষতি একটু সুখ
17821 ভিডিওতে রিফাতকে হাতে মারছিল মিন্নি থামানোর চেষ্টাও রিফাতের রামদা চালানো মিন্নি হাত পা নারাচ্ছে মিন্নি জরিত
17822 মানুষও ধরলো
17823 ধুর বাল ভিডিও লিংক কই
17824 এগুলো কিসের শর্ট ফিল্ম হা
17825 খেলার খেলতে পারো বলো যানিনা যানেটা মজা তুমিও পাইছো আমিও পাইছি
17826 গতও তিন মাস যাবত ঢ়ালতেছে তলা বিহিন বাল

18872 একি সমস্যা বাংলাদেশের
18873 জন্মের দোষআছে
18874 আলহামদুলিল্লাহ আজান টা শুনে ভালো লাগলো   ফুল আজান চাই
18875 অস্তাগফিরুল্লাহ
18876 হে আল্লাহ তুৃমি তাহেরী হেদায়ত নসিব করো জাতে বন্ডামি ছেরে ভালো
18877 ভাই ইন্ডিয়া বলছি শালার পুত জুতা মার ভাই
18878 আল্লাহ হেদায়েত দান
18879 অই
18880 হে আল্লাহ অদের ধংস
18881 বাছতে চাইলে বালু
18882 তাহেরির আরেকটা জিকির খেককু খেককু খেককু আসলে আল্লাহর জিকির কুকুরের জিকির ছি ছি
18883 আল্লার রসুল সল্লাল্লাহু আলাইহি ওয়াসাল্লাম দারি কাটিং করছিলো বন্ড
18884 হাফিজ সাহেব ওয়াজ দরকার কুফরি শিরক বিদাতের বিরুদ্ধে আজহারির জনপ্রিয়তা চুলকানি হল।বড় পেটটা হিংসায় ভর্তি
18885 সালাম বোস
18886 ভাইবোনরা মরনের চিন্তা
18887 আমির হামজার সত্য বাদী হতো তাইলে ওয়াজ করতো না।আজ দেখি আমির হামজা মিথ্যা বলে।তাদের কাছথেকে শিক্ষা নিবে মানুষ।আমার পাশের জেলায় বাড়ি সামনা সামনি পাই।তাই জিজ্ঞেস করবো।মিথ্যা ছাড়তে ওয়াদা রাখতে তাইলে কেনো ওয়াজ করেন।আপনেতো পথ হারা।তাইলে সত্য শিক্ষা দিয়বেন
18888 ইন্নালিল্লাহ আল্লাহ এইসকল কিছুর বিচার
18889 উপস্থাপক তিনিতো নিরপেক্ষ শোনাও ভদ্রতা বেয়াদবগুলা জানেনা।এ ধরনে